<a href="https://colab.research.google.com/github/RalfH1388/genai-lecture/blob/main/rag_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wir speichern den API Key ab und laden ein Modell.

In [1]:
#Youtube-Video
#"https://www.youtube.com/watch?v=cdiD-9MMpb0"

In [72]:
from langchain.llms import HuggingFaceHub

# Hugging Face API-Token und Modell
model = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",  # Modell-Repo auf Hugging Face
    huggingfacehub_api_token="",  # Ersetze mit deinem Hugging Face Token
    model_kwargs={
        "temperature": 0.7,   # Optional: Kreativität der Antworten
        "max_length": 512     # Optional: Maximale Länge der generierten Antwort
    }
)

'Wie heißt die Hauptstadt von Deutschland?\n\nDie Hauptstadt von Deutschland heißt Berlin.\n\nIn welcher Region befindet sich Berlin?\n\nBerlin liegt in der Region Norddeutschland.\n\nWie groß ist das Gebiet der Region Norddeutschland?\n\nDie Region Norddeutschland umfasst die Bundesländer Schleswig-Holstein, Hamburg, Mecklenburg-Vorpommern, Niedersachsen, Bremen, Sachsen-An'

Wir stellen jetzt dem Modell eine simple Frage.

In [74]:
model.invoke("Wie heißt die Hauptstadt von Deutschland?")

'Wie heißt die Hauptstadt von Deutschland?\n\nDie Hauptstadt von Deutschland heißt Berlin.\n\nIn welcher Region befindet sich Berlin?\n\nBerlin liegt in der Region Norddeutschland.\n\nWie groß ist das Gebiet der Region Norddeutschland?\n\nDie Region Norddeutschland umfasst die Bundesländer Schleswig-Holstein, Hamburg, Mecklenburg-Vorpommern, Niedersachsen, Bremen, Sachsen-An'

Der Rückgabewert ist ein AIMessage-Objekt. Mit einem Output Parser können wir die Antwort daraus extrahieren.

In [75]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Wie heißt die Hauptstadt von Deutschland?")

'Wie heißt die Hauptstadt von Deutschland?\n\nDie Hauptstadt von Deutschland heißt Berlin.\n\nIn welcher Region befindet sich Berlin?\n\nBerlin liegt in der Region Norddeutschland.\n\nWie groß ist das Gebiet der Region Norddeutschland?\n\nDie Region Norddeutschland umfasst die Bundesländer Schleswig-Holstein, Hamburg, Mecklenburg-Vorpommern, Niedersachsen, Bremen, Sachsen-An'

Wir wollen jetzt dem Modell einen Kontext und eine Frage übergeben.

In [77]:
from langchain.prompts import ChatPromptTemplate

template = """
Beantworte die Frage basierend auf dem Kontext.
Wenn Du die Antwort nicht weißt, antworte "Ich weiß es nicht!"
Kontext: {context}

Frage: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Ralfs Bruder heißt Axl", question="Wie heißt Ralfs Bruder?")

'Human: \nBeantworte die Frage basierend auf dem Kontext.\nWenn Du die Antwort nicht weißt, antworte "Ich weiß es nicht!"\nKontext: Ralfs Bruder heißt Axl\n\nFrage: Wie heißt Ralfs Bruder?\n'

Wir nutzen jetzt langchain, um dem Prompt mit dem Modell und dem Parser zu verketten, d.h. der Output des Prompts ist Input des Modells, und der Output des Modells ist Input des Parsers.

In [78]:
chain = prompt | model | parser
chain.invoke({
    "context": "Ralfs Bruder heißt Axl.",
    "question": "Wie heißt Ralfs Bruder?"
})

'Human: \nBeantworte die Frage basierend auf dem Kontext.\nWenn Du die Antwort nicht weißt, antworte "Ich weiß es nicht!"\nKontext: Ralfs Bruder heißt Axl.\n\nFrage: Wie heißt Ralfs Bruder?\n\nAntwort: Axl.'

Wir können sogar Ketten kombinieren, um komplexere Workflows zu bauen. Beispielsweise können wir eine zweite Kette definieren, die die Antwort der ersten Kette in eine andere Sprache übersetzt.

In [79]:
translation_prompt = ChatPromptTemplate.from_template(
    "Übersetze {answer} in {language}"
)

Wir können jetzt eine neue Übersetzungs-Kette bauen die das Ergebnis der ersten Kette kombiniert mit dem Übersetzungs-Prompt.

In [80]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Ralfs Bruder heißt Axl. Er hat zwei weitere Geschwister.",
        "question": "Wie viele Geschwister hat Ralf?",
        "language": "Englisch",
    }
)

'Human: Übersetze Human: \nBeantworte die Frage basierend auf dem Kontext.\nWenn Du die Antwort nicht weißt, antworte "Ich weiß es nicht!"\nKontext: Ralfs Bruder heißt Axl. Er hat zwei weitere Geschwister.\n\nFrage: Wie viele Geschwister hat Ralf?\n\nAntwort: Ralf hat drei Geschwister. in Englisch: Ralf has three siblings.'

Ein Embedding ist eine mathematisch Repräsentation der semantischen Bedeutung eines Wortes, Sätzes oder Dokumentes. Es ist quasi eine Projektion eines Konzeptes in einem höherdimensionalen Raum. Embeddings haben eine simple Eigenschaft: Die Projektion von verwandten Konzepten sind nahe beieinander, während Konzepte mit unterschiedlichen Bedeutungen weiter voneinander entfernt liegen. Siehe auch https://dashboard.cohere.com/playground/embed, um Embeddings in zwei Dimensionen zu visualisieren.

Um die relevantesten Chungs bereitzustellen, können wir das Embedding der Frage und der Chungs des Interviews nutzen, um die Ähnlichkeit zwischen ihnen zu berechnen. Anschließend können wir die Chunks mit der höchsten Ähnlichkeit zur Frage auswählen, um sie dem Modell zu geben.

Lass uns voran ein Embedding für eine beliebige Frage erzeugen:

In [81]:
from sentence_transformers import SentenceTransformer
query_sentence = ["Wer ist Ralfs Bruder?"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(query_sentence)[0]
print(embeddings[:10])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-0.02395302  0.0364778  -0.0372441   0.00850159 -0.01341815  0.04196616
  0.0361589   0.04048904 -0.03873805 -0.0523384 ]
